In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from preprocessing import Processing_DataFrame
from training import TRAINING_CNN_BLSTM
from utils_function import *

In [13]:
data_train = pd.read_excel('data/ner_dataset_train.xlsx')
data_train = data_train.dropna().reset_index(drop=True)

data_test = pd.read_excel('data/ner_dataset_test.xlsx')
data_test = data_test.dropna().reset_index(drop=True)

print(data_train.shape, data_test.shape)
data_train.head()

(35818, 3) (11941, 3)


,Intent,Parameters,Question
0,B-gpe,"[[0, 7, ""B-gpe""]]",Israeli police investigating a money-launderin...
1,B-perI-perB-orgI-orgB-tim,"[[0, 7, ""B-per""], [8, 14, ""I-per""], [32, 42, ""...",Senator McCain could secure the Republican Par...
2,B-gpeB-tim,"[[0, 8, ""B-gpe""], [58, 64, ""B-tim""]]",American consumers fear prices will go even hi...
3,B-gpeB-gpeB-tim,"[[4, 10, ""B-gpe""], [35, 42, ""B-gpe""], [134, 13...",The Syrian newspaper is calling on Israeli off...
4,B-geoB-geoI-geoB-perB-gpeB-orgB-tim,"[[29, 32, ""B-geo""], [111, 120, ""B-geo""], [33, ...",Despite her absence from the New Zealand event...


In [14]:
#Preprocess Train set
data_train = Processing_DataFrame(data_train)
data_train.remove_label(labels=['Tag','B-art', 'B-eve', 'B-nat', 'I-art', 'I-eve', 'I-nat'])
data_train.remove_higher_len(len_threshold=40)
list_labels = data_train.preprocessing_ner()
data_train.create_target(list_labels)
data_train.split_word_char_label()


#Preprocess Test set
data_test = Processing_DataFrame(data_test)
data_test.remove_label(labels=['Tag','B-art', 'B-eve', 'B-nat', 'I-art', 'I-eve', 'I-nat'])
data_test.remove_higher_len(len_threshold=40)
_ = data_test.preprocessing_ner()
data_test.create_target(list_labels)
data_test.split_word_char_label()

In [15]:
data_train.head()

,Intent,Parameters,Question,Preprocess_Question,Target,input
0,B-gpe,"[[0, 7, ""B-gpe""]]",Israeli police investigating a money-launderin...,B-gpe police investigating a money-laundering ...,B-gpe O O O O O O O O O O O O O O O,"[[Israeli, [I, s, r, a, e, l, i], B-gpe], [pol..."
1,B-perI-perB-orgI-orgB-tim,"[[0, 7, ""B-per""], [8, 14, ""I-per""], [32, 42, ""...",Senator McCain could secure the Republican Par...,B-per I-per could secure the B-org I-org nomin...,B-per I-per O O O B-org I-org O O O B-tim,"[[Senator, [S, e, n, a, t, o, r], B-per], [McC..."
2,B-gpeB-tim,"[[0, 8, ""B-gpe""], [58, 64, ""B-tim""]]",American consumers fear prices will go even hi...,B-gpe consumers fear prices will go even highe...,B-gpe O O O O O O O O O B-tim O O O O,"[[American, [A, m, e, r, i, c, a, n], B-gpe], ..."
3,B-gpeB-gpeB-tim,"[[4, 10, ""B-gpe""], [35, 42, ""B-gpe""], [134, 13...",The Syrian newspaper is calling on Israeli off...,The B-gpe newspaper is calling on B-gpe offici...,O B-gpe O O O O B-gpe O O O O O O O O O O O O ...,"[[The, [T, h, e], O], [Syrian, [S, y, r, i, a,..."
4,B-geoB-geoI-geoB-perB-gpeB-orgB-tim,"[[29, 32, ""B-geo""], [111, 120, ""B-geo""], [33, ...",Despite her absence from the New Zealand event...,Despite her absence from the B-geo I-geo event...,O O O O O B-geo I-geo O O O O O O O B-per O O ...,"[[Despite, [D, e, s, p, i, t, e], O], [her, [h..."


In [16]:
from tensorflow.keras.optimizers.legacy import Nadam

EPOCHS = 100                # paper: 80
DROPOUT = 0.5               # paper: 0.68
DROPOUT_RECURRENT = 0       # not specified in paper
LSTM_STATE_SIZE = 250       # paper: 275
CONV_SIZE = 3               # paper: 3
LEARNING_RATE = 0.015       # paper 0.0105
OPTIMIZER = Nadam()         # paper uses SGD(lr=self.learning_rate), Nadam() recommended
MAX_LEN = 40  

DATA_TRAIN = data_train
DATA_TEST = data_test

In [17]:
model = TRAINING_CNN_BLSTM(EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, MAX_LEN, DATA_TRAIN, DATA_TEST)
model.embedding()
model.createBatches()

In [18]:
model.build_model(print_summary=True, save_model_image=True)

Model is built. Image of model is saved completely
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Character_input (InputLayer)   [(None, None, 40)]   0           []                               
                                                                                                  
 Character_embedding (TimeDistr  (None, None, 40, 30  2910       ['Character_input[0][0]']        
 ibuted)                        )                                                                 
                                                                                                  
 dropout (Dropout)              (None, None, 40, 30  0           ['Character_embedding[0][0]']    
                                )                                                                 
                                           

In [19]:
model.train_model(name='finalModel_100epochs')

Epoch 0/100
Macro F1 Score: 0.175935821512653 - Testing by sampling 1000 sentences randomly from test data.
Epoch 10/100
Macro F1 Score: 0.6903463737691585 - Testing by sampling 1000 sentences randomly from test data.
Epoch 20/100
Macro F1 Score: 0.7118618048258527 - Testing by sampling 1000 sentences randomly from test data.
Epoch 30/100
Macro F1 Score: 0.7583567477460774 - Testing by sampling 1000 sentences randomly from test data.
Epoch 40/100
Macro F1 Score: 0.7939785437823612 - Testing by sampling 1000 sentences randomly from test data.
Epoch 50/100
Macro F1 Score: 0.8367675262313665 - Testing by sampling 1000 sentences randomly from test data.
Epoch 60/100
Macro F1 Score: 0.7806385806819932 - Testing by sampling 1000 sentences randomly from test data.
Epoch 70/100
Macro F1 Score: 0.8155370386440385 - Testing by sampling 1000 sentences randomly from test data.
Epoch 80/100
Macro F1 Score: 0.7541180152915244 - Testing by sampling 1000 sentences randomly from test data.
Epoch 90/100

In [20]:
model.evaluate_model()

Model is evaluated on test set which has 11564 sentences.
------------------------------------------------------------
Predicting in progress ___ 11564 sentences remaining.
Predicting in progress ___ 10064 sentences remaining.
Predicting in progress ___ 8564 sentences remaining.
Predicting in progress ___ 7064 sentences remaining.
Predicting in progress ___ 5564 sentences remaining.
Predicting in progress ___ 4064 sentences remaining.
Predicting in progress ___ 2564 sentences remaining.
Predicting in progress ___ 1064 sentences remaining.
------------------------------------------------------------
              precision    recall  f1-score   support

       B-geo       0.80      0.92      0.85      8720
       B-gpe       0.88      0.93      0.90      3657
       B-org       0.77      0.66      0.71      4706
       B-per       0.82      0.82      0.82      3936
       B-tim       0.85      0.89      0.87      4854
       I-geo       0.75      0.80      0.77      1740
       I-gpe   